In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
df = pd.read_csv('../../data/preprocessed_data.csv')
df.head()

,movie,plot,genres
0,"""#7DaysLater"" (2013)",days later interactive comedy series featuring...,['Comedy']
1,"""#Cake"" (2015)",cake hour long serial narrative comedy manhunt...,['Comedy']
2,"""#DaddyLeaks"" (????)",life four close friends late thirties change f...,['Comedy']
3,"""#Elmira"" (2014)",elmira follows story bunch strangers respond c...,['Comedy']
4,"""#Fuga"" (2016)",months apocalyptic event group survivors find ...,"['Action', 'Drama', 'Horror']"


In [6]:

import numpy as np 
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
# from bs4 import BeautifulSoup
# import plotly.graph_objs as go
# import plotly as py
# import cufflinks
from IPython.core.interactiveshell import InteractiveShell
# import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
# from plotly.offline import iplot
# cufflinks.go_offline()
# cufflinks.set_config_file(world_readable=True, theme='pearl')

In [7]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


2023-11-18 18:25:57.249776: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9631 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:02:00.0, compute capability: 7.5


In [8]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

df['plot'] = df['plot'].apply(clean_text)
df['plot'] = df['plot'].str.replace('\d+', '')

/tmp/ipykernel_1989/3028245631.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df['plot'] = df['plot'].str.replace('\d+', '')


In [9]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 60000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['plot'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 132110 unique tokens.


In [10]:
X = tokenizer.texts_to_sequences(df['plot'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
# X = X[:1000]
print('Shape of data tensor:', X.shape)

Shape of data tensor: (259557, 250)


In [11]:
Y = np.load('../../vectorised_data/y.npy')
# Y = Y[:1000]
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (259557, 20)


In [12]:
# Oversample this data for better results
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler(random_state=42)
# X, Y = ros.fit_resample(X, Y)

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(207645, 250) (207645, 20)
(51912, 250) (51912, 20)


In [14]:
print(X_train.shape[1])

250


In [15]:
import os
os.environ['WANDB_NOTEBOOK_NAME'] = 'RNN_scratch_aayush.ipynb'


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, Bidirectional, LSTMCell, RNN, Dense, GRUCell, LayerNormalization, GRU
from tensorflow.keras.callbacks import EarlyStopping
# import matplotlib.pyplot as plt
# import wandb
# from wandb.keras import WandbCallback

# wandb.init(project='Reddit_mental_RNN')

# Your data: X_train, X_val, X_test, y_train_encoded, y_val_encoded, y_test_encoded

# Manual specification of parameters for the first run
params = {
    'units': 128,
    'dropout': 0,
    'layers': 2,
    'batch_size': 64,
    'epochs': 5,
    'lr': 0.001,
}


In [17]:
def hit_rate(predicted, true):
    # Use element-wise logical AND to check if at least one class is predicted correctly
    correct_predictions = np.any(np.logical_and(predicted, true), axis=1)
    
    # Calculate the hit rate as the percentage of correct samples
    hit_rate = (np.sum(correct_predictions) / len(correct_predictions))
    
    return hit_rate

In [20]:
from sklearn.metrics import jaccard_score, classification_report

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(params['dropout']))
for i in range(params['layers']):
    model.add(GRU(params['units'], return_sequences=i != params['layers']-1, recurrent_dropout=params['dropout']))
    model.add(LayerNormalization())
model.add(Dense(20, activation='sigmoid'))
optimizer = tf.keras.optimizers.Adam(learning_rate=params['lr'])
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[tf.keras.metrics.CategoricalAccuracy()])
model.summary()

epochs = params['epochs']
batch_size = params['batch_size']

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 100)          6000000   
                                                                 
 spatial_dropout1d_1 (Spati  (None, 250, 100)          0         
 alDropout1D)                                                    
                                                                 
 gru_2 (GRU)                 (None, 250, 128)          88320     
                                                                 
 layer_normalization_2 (Lay  (None, 250, 128)          256       
 erNormalization)                                                
                                                                 
 gru_3 (GRU)                 (None, 128)               99072     
                                                                 
 layer_normalization_3 (Lay  (None, 128)              

KeyboardInterrupt: 

In [ ]:

predictions = model.predict(X_test)

# print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
print(predictions[:5])
predictions = np.round(predictions)
print(predictions[:5])
print(Y_test[:5])
print('Jaccard Score: {:.4f}'.format(jaccard_score(Y_test, predictions, average='samples')))
print(f'Hit Rate: {hit_rate(predictions, Y_test):.4f}')
print('Classification Report:')
print(classification_report(Y_test, predictions, zero_division=True))

# wandb.log({'test_loss': accr[0], 'test_acc': accr[1]})
# wandb.log({'train_loss': history.history['loss'], 'val_loss': history.history['val_loss']})
# wandb.log({'train_acc': history.history['accuracy'], 'val_acc': history.history['val_accuracy']})


# plt.title('Accuracy')
# plt.plot(history.history['accuracy'], label='train')
# plt.plot(history.history['val_accuracy'], label='test')
# plt.legend()
# plt.show()


In [ ]:
# Plot the confusion matrix
from sklearn.metrics import ConfusionMatrixDisplay, multilabel_confusion_matrix, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


# plot the multilabel confusion matrix
cm = multilabel_confusion_matrix(Y_test, predictions)

# plot the confusion matrix
# cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# plot the confusion matrix
f, axes = plt.subplots(4, 5, figsize=(25, 15))
axes = axes.ravel()
for i in range(20):
    disp = ConfusionMatrixDisplay(confusion_matrix(Y_test[:, i],
                                                   predictions[:, i]),
                                  display_labels=[0, 1])
    disp.plot(ax=axes[i], values_format='.4g')
    disp.ax_.set_title(f'class {i}')
    if i<10:
        disp.ax_.set_xlabel('')
    if i%5!=0:
        disp.ax_.set_ylabel('')
    disp.im_.colorbar.remove()

plt.subplots_adjust(wspace=0.10, hspace=0.1)
f.colorbar(disp.im_, ax=axes)
plt.show();
# print(cm)


In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(cm, annot=True, fmt=".0f", linewidths=.5, square = True, cmap = 'cividis')
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.title('Confusion matrix')
plt.show();

In [ ]:
del model